In [11]:
#Library Import

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import altair as alt
import scipy.stats as stats
from scipy.stats import norm

# import seaborn as sns
# from sklearn.neighbors import KernelDensity
# from bokeh.models import ColumnDataSource, Label, PrintfTickFormatter
# from bokeh.palettes import Dark2_5 as colors
# from bokeh.plotting import figure, show
# from bokeh.sampledata.cows import data as df
# from bokeh.io import output_notebook


In [12]:
#copied from main.ipynb
# #import pandas as pd


df_course = pd.read_csv('assets/student.course.csv')
df_record = pd.read_csv('assets/student.record.csv')
df_term = pd.read_table('assets/term.table.txt', delimiter="\t").fillna('Unknown')



df_term.columns = ['TERM_ID', 'TERM_DESCRIPTION']

terms = ['Fall', 'Winter', 'Summer', 'Spring', 'Unknown']
df_term['TERM_NAME'] = df_term['TERM_DESCRIPTION'].apply(lambda x: list(filter(lambda y: y!='', [t if t in x else '' for t in terms]))[0])
df_term['TERM_YEAR'] = df_term['TERM_DESCRIPTION'].apply(lambda x: x[-4:] if x[-4:].isnumeric() else None)

merge_course_df = pd.merge(
                    df_course,
                    df_term, 
                    how='left',
                    left_on='TERM',
                    right_on='TERM_ID')

terms_student = ['ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM']
merge_record_df = df_record.copy()
for term in terms_student:
    
    merge_record_df = pd.merge(
                        merge_record_df,
                        df_term, 
                        how='left',
                        left_on=term,
                        right_on='TERM_ID')
    
    

    merge_record_df.rename(columns={'TERM_ID': term + '_' + 'TERM_ID', 
                            'TERM_DESCRIPTION': term + '_' + 'TERM_DESCRIPTION', 
                            'TERM_NAME': term + '_' + 'TERM_NAME', 
                            'TERM_YEAR': term + '_' + 'TERM_YEAR'}, inplace=True)    
    
 


# df_term
# df_course
# df_record
# merge_course_df
merge_record_df

    
course_columns = ['ANONID', 'SUBJECT', 'CATALOG_NBR', 'GRD_PTS_PER_UNIT', 'GPAO', 'DIV',
                  'ANON_INSTR_ID', 'TERM', 'TERM_ID', 'TERM_DESCRIPTION', 'TERM_NAME', 'TERM_YEAR']

record_columns = ['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA', 'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE',
                  'LAST_ACT_READ_SCORE', 'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE', 
                  'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1', 'STDNT_GROUP2', 'MAJOR1_DEPT', 
                  'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID', 'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM', 
                  'MAJOR3_TERM_TERM_ID', 'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME', 'MAJOR3_TERM_TERM_YEAR']   

# merge_course_df.describe()
# merge_record_df.describe()


# https://www.bls.gov/oes/tables.htm
# https://data.census.gov/table/ACSST1Y2015.S0101
# https://data.census.gov/table/ACSST1Y2015.S1501?t=Education&g=040XX00US06
# merge_record_df.sort_values(by=['MAJOR3_TERM_TERM_YEAR'], ascending=[True])['MAJOR3_TERM_TERM_YEAR'].unique()
# 1992 - 2015


# Occupational data by gender
# https://www.bls.gov/cps/cps_aa2015.htm

# https://www.axios.com/2020/06/30/half-us-population-unemployed
# the labor market is only at 50%?


merge_record_df[['ADMIT_TERM','ADMIT_TERM_TERM_ID','ADMIT_TERM_TERM_DESCRIPTION','ADMIT_TERM_TERM_NAME']]


,ADMIT_TERM,ADMIT_TERM_TERM_ID,ADMIT_TERM_TERM_DESCRIPTION,ADMIT_TERM_TERM_NAME
0,NaN,NaN,NaN,NaN
1,110.0,110.0,Fall 2008,Fall
2,63.0,63.0,Fall 1998,Fall
3,106.0,106.0,Fall 2007,Fall
4,83.0,83.0,Fall 2002,Fall
...,...,...,...,...
138883,NaN,NaN,NaN,NaN
138884,126.0,126.0,Fall 2012,Fall
138885,98.0,98.0,Fall 2005,Fall
138886,NaN,NaN,NaN,NaN


In [13]:
merge_record_df.columns

Index(['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA',
       'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE', 'LAST_ACT_READ_SCORE',
       'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE',
       'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1',
       'STDNT_GROUP2', 'MAJOR1_DEPT', 'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID',
       'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM',
       'ADMIT_TERM_TERM_ID', 'ADMIT_TERM_TERM_DESCRIPTION',
       'ADMIT_TERM_TERM_NAME', 'ADMIT_TERM_TERM_YEAR', 'MAJOR1_TERM_TERM_ID',
       'MAJOR1_TERM_TERM_DESCRIPTION', 'MAJOR1_TERM_TERM_NAME',
       'MAJOR1_TERM_TERM_YEAR', 'MAJOR2_TERM_TERM_ID',
       'MAJOR2_TERM_TERM_DESCRIPTION', 'MAJOR2_TERM_TERM_NAME',
       'MAJOR2_TERM_TERM_YEAR', 'MAJOR3_TERM_TERM_ID',
       'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME',
       'MAJOR3_TERM_TERM_YEAR'],
      dtype='object')

In [14]:
table = pd.pivot_table(merge_record_df, values='ANONID', index=['ADMIT_TERM_TERM_YEAR','SEX'],
                       aggfunc="count")
table.reset_index(inplace = True)
#display(table)
area = alt.Chart(table).mark_area(opacity = 0.4).encode(
        x="ADMIT_TERM_TERM_YEAR",
        y=alt.Y('ANONID', stack = False),
        color="SEX",
        ).properties(title={"text": "number of students at university X per year grouped by sex"}
        )
area

alt.Chart(...)

In [21]:

result = merge_course_df.groupby('ANONID').agg(
    avg_GPAO=('GPAO', 'mean'),
    max_TERM_YEAR=('TERM_YEAR', 'max')
).reset_index()

result.head()
merge_df = pd.merge(
                    merge_record_df,
                    result, 
                    how='left',
                    left_on='ANONID',
                    right_on='ANONID')
#merge_df.head()

In [20]:
columnlst = ['HSGPA', 'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE', 'LAST_ACT_READ_SCORE',
       'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE',
       'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE','avg_GPAO']

def getFigs(timeCol, groupCol, columns, dataframe, agg='mean'):
    def layered_area_chart(col, data):
        area = alt.Chart(data).mark_area(opacity=0.4).encode(
            x=alt.X(f"{timeCol}:T", title="Year"),
            y=alt.Y(col, stack=False),
            color=alt.Color(groupCol, title="Sex")
        ).properties(
            title=f"{agg.capitalize()} {col} per year grouped by sex"
        )
        return area

    for col in columns:
        table = pd.pivot_table(dataframe, values=col, index=[timeCol, groupCol], aggfunc=agg)
        table.reset_index(inplace=True)
        
        # Generate and return chart (in Jupyter, returning should display the chart)
        fig = layered_area_chart(col, table)
        display(fig)  # Use display() in Jupyter environments

# Call the function to generate the charts
getFigs('ADMIT_TERM_TERM_YEAR', 'SEX', columnlst, merge_df)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [22]:
merge_df.columns

Index(['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA',
       'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE', 'LAST_ACT_READ_SCORE',
       'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE',
       'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1',
       'STDNT_GROUP2', 'MAJOR1_DEPT', 'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID',
       'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM',
       'ADMIT_TERM_TERM_ID', 'ADMIT_TERM_TERM_DESCRIPTION',
       'ADMIT_TERM_TERM_NAME', 'ADMIT_TERM_TERM_YEAR', 'MAJOR1_TERM_TERM_ID',
       'MAJOR1_TERM_TERM_DESCRIPTION', 'MAJOR1_TERM_TERM_NAME',
       'MAJOR1_TERM_TERM_YEAR', 'MAJOR2_TERM_TERM_ID',
       'MAJOR2_TERM_TERM_DESCRIPTION', 'MAJOR2_TERM_TERM_NAME',
       'MAJOR2_TERM_TERM_YEAR', 'MAJOR3_TERM_TERM_ID',
       'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME',
       'MAJOR3_TERM_TERM_YEAR', 'avg_GPAO', 'max_TERM_YEAR'],
      dtype='object')

In [29]:
merge_df[['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR']].head()

,MAJOR3_DESCR,MAJOR2_DESCR,MAJOR1_DESCR
0,NaN,NaN,NaN
1,NaN,NaN,Asian Studies BA
2,NaN,NaN,Psychology BA
3,NaN,NaN,Civil Engineering BSE
4,NaN,NaN,NaN


In [30]:
merge_df['MAJOR1_DESCR'].unique()

array([nan, 'Asian Studies BA', 'Psychology BA', 'Civil Engineering BSE',
       'American Culture BA', 'Anthropology BA',
       'Chemical Engineering BSE', 'Environ Policy & Behavior BS',
       'Communication BA', 'Computer Science BSE',
       'Industrial & Oper Eng BSE', 'Mechanical Engineering BSE',
       'General Studies BGS', 'Architecture BS', 'Mathematics BS',
       "Women's Studies BA", 'Computer Engineering BSE', 'History BA',
       'Brain,Behavior & Cognit Sc BS', 'English BA', 'Psychology BS',
       'History of Art BA', 'Spanish BA', 'Sociology BA', 'Economics BA',
       'General Biology BS', 'Aerospace Engineering  BSE',
       'Film and Video Studies BA', 'Nursing BS Fresh',
       'Electrical Engineering BSE', 'Biology BS',
       'International Studies BA', 'Chemistry BSChem',
       'Political Science BA', 'Business Administration BBA',
       'Microbiology BS', 'Individualized Concentrat BA',
       'Cellular & Molec Biology BS', 'Graphic Design BFA',
       'A

In [96]:
list1=[]
def getMajorType(x):
    
    if type(x) == str:
        lst = x.split(" ")
        deg = lst[-1]
        if lst[-1].startswith('B'):
            return lst[-1]
        elif lst[-1] == 'ABEd':
            return lst[-1]
        elif "BS" in lst:
            return 'BS'
        elif "BSE" in lst:
            return 'BSE'
        elif 'Nursing' in lst:
            return 'BSN'
        elif 'BFA' in lst:
            return 'BFA'
        else:
            list1.append(lst)
            return 'other'

merge_df["Deg_Type"] = merge_df['MAJOR1_DESCR'].apply(lambda x: getMajorType((x)))
merge_df['Deg_Type'].unique()
list2 = []
for x in list1:
    if x not in list2:
        list2.append(x)

for l in list2:
    print(' '.join(l))

len(list1)

Arab,Arm,Pers,Turk & Islamic S
Program in Entrepreneurship
Sustainability Scholars Prog
Earth System Science
Honors in the Liberal Arts
Unclassified


317

In [97]:
merge_df['Deg_Type'].unique()

array([None, 'BA', 'BSE', 'BS', 'BGS', 'BSChem', 'BBA', 'BFA', 'BMus',
       'other', 'BSEd', 'ABEd', 'BMusA', 'BSN', 'BDA', 'BTA', 'BFADes',
       'BioMed', 'BMU', 'BioE', 'BFM'], dtype=object)